# Video Scraping from Links
- This notebook is for when we have collected labels and links already

- Downloading all videos takes a long time, so we will only scrape selected videos

- With this notebook, we can use `download_videos.py` to download all videos in a metadata.csv to create our raw combined dataset, after we have decided our target words

In [1]:
import pathlib
import os
import pandas as pd
import requests
import download_videos as dv

In [2]:
# change working directory to the project root directory
current_dir = os.getcwd()
os.chdir(current_dir + '/../../')
# this should be the project root directory
os.getcwd()

'/home/ben/projects/SaoPauloBrazilChapter_BrazilianSignLanguage'

In [3]:
metadata_df = pd.read_csv('data/raw/combined/metadata_combined.csv')

## *for creating our combined raw dataset, after we have decided our target words*

Download all videos for a collection of words from all data sources

define list of target words 

In [4]:
metadata_df = metadata_df.sort_values(['label', 'data_source']).reset_index(drop=True)

In [5]:
sample_words = ['abacaxi', 'banana', 'café', 'carne', 'cebola']
target_metadata_df = metadata_df[metadata_df.label.isin(sample_words)].sort_values(['label', 'data_source']).reset_index(drop=True)
target_metadata_df.label.value_counts()

label
banana    6
café      6
carne     6
cebola    6
Name: count, dtype: int64

download videos

In [6]:
output_path = os.path.join('data', 'raw', 'combined', 'videos')
verify_ssl_settings = {
    'ne': True,
    'vl': True,
    'sb': False,
    'uf': True
}

In [8]:
dv.download_videos_from_metadata(
    metadata=target_metadata_df,
    output_path=output_path,
    verify_ssl_settings=verify_ssl_settings,
    verbose=True
    )

	Video successfully downloaded to data/raw/combined/videos/banana_ne_1.mp4


/home/ben/projects/SaoPauloBrazilChapter_BrazilianSignLanguage/.venv/lib/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'videos.nals.cce.ufsc.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


	Video successfully downloaded to data/raw/combined/videos/banana_sb_2.mp4
	Video successfully downloaded to data/raw/combined/videos/banana_uf_3.mp4
	Video successfully downloaded to data/raw/combined/videos/banana_vl_4.mp4
	Video successfully downloaded to data/raw/combined/videos/banana_vl_5.mp4
	Video successfully downloaded to data/raw/combined/videos/banana_vl_6.mp4
	Video successfully downloaded to data/raw/combined/videos/café_ne_7.mp4


/home/ben/projects/SaoPauloBrazilChapter_BrazilianSignLanguage/.venv/lib/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'videos.nals.cce.ufsc.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


	Video successfully downloaded to data/raw/combined/videos/café_sb_8.mp4
	Video successfully downloaded to data/raw/combined/videos/café_uf_9.mp4
	Video successfully downloaded to data/raw/combined/videos/café_vl_10.mp4
	Video successfully downloaded to data/raw/combined/videos/café_vl_11.mp4
	Video successfully downloaded to data/raw/combined/videos/café_vl_12.mp4
	Video successfully downloaded to data/raw/combined/videos/carne_ne_13.mp4


/home/ben/projects/SaoPauloBrazilChapter_BrazilianSignLanguage/.venv/lib/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'videos.nals.cce.ufsc.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


	Video successfully downloaded to data/raw/combined/videos/carne_sb_14.mp4
	Video successfully downloaded to data/raw/combined/videos/carne_uf_15.mp4
	Video successfully downloaded to data/raw/combined/videos/carne_vl_16.mp4
	Video successfully downloaded to data/raw/combined/videos/carne_vl_17.mp4
	Video successfully downloaded to data/raw/combined/videos/carne_vl_18.mp4
	Video successfully downloaded to data/raw/combined/videos/cebola_ne_19.mp4


/home/ben/projects/SaoPauloBrazilChapter_BrazilianSignLanguage/.venv/lib/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'videos.nals.cce.ufsc.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


	Video successfully downloaded to data/raw/combined/videos/cebola_sb_20.mp4
	Video successfully downloaded to data/raw/combined/videos/cebola_uf_21.mp4
	Video successfully downloaded to data/raw/combined/videos/cebola_vl_22.mp4
	Video successfully downloaded to data/raw/combined/videos/cebola_vl_23.mp4
	Video successfully downloaded to data/raw/combined/videos/cebola_vl_24.mp4


## *for collecting metadata from the videos*

In [24]:
video_metadata = dv.collect_metadata_from_directory('data/raw/combined/videos')

In [25]:
video_metadata[0]

{'filename': 'banana_ne_1.mp4',
 'frame_count': 51,
 'fps': 12.0,
 'width': 240,
 'height': 176,
 'duration_sec': 4}

In [26]:
# check for None values
for i,d in enumerate(video_metadata):
    if d is None:
        print(i)
# should be no prints

In [29]:
metadata_df

,label,data_source,video_url,sign_id
0,ajudar,ne,https://www.ines.gov.br/dicionario-de-libras/p...,2
1,ajudar,sb,https://videos.nals.cce.ufsc.br/SignBank/V%C3%...,1
2,ajudar,uf,https://sistemas.cead.ufv.br/capes/dicionario/...,0
3,ajudar,vl,https://libras.cin.ufpe.br/storage/videos/2021...,3
4,ajudar,vl,https://libras.cin.ufpe.br/storage/videos/2021...,2
...,...,...,...,...
152,vagina,sb,https://videos.nals.cce.ufsc.br/SignBank/V%C3%...,1
153,vagina,uf,https://sistemas.cead.ufv.br/capes/dicionario/...,0
154,vagina,vl,https://libras.cin.ufpe.br/storage/videos/2021...,3
155,vagina,vl,https://libras.cin.ufpe.br/storage/videos/2021...,2


In [31]:
video_metadata_df = pd.DataFrame(video_metadata)
video_metadata_df = pd.concat([target_metadata_df, video_metadata_df], axis=1)

In [34]:
video_metadata_df[[
    'filename',
    'label',
    'data_source',
    # 'signer_number',
    'frame_count',
    'fps',
    'duration_sec',
    'width',
    'height'
]].to_csv('data/raw/combined/target_dataset_video_metadata.csv', index=False)